In [1]:
# load predictors (X_df), outcomes_df (success/fail, later try regression)
# run cross-validated random forests, training, test, metrics
# then extend to other models

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import time
import pymysql as mdb
import re

In [3]:
%matplotlib qt

In [4]:
from prep_predictors_for_ml import *
%load_ext autoreload
%autoreload 2

In [13]:
saveas_path = '/Users/jamie/insight data/figures/'

In [23]:
X_df = pd.read_pickle('X_df.save')

In [24]:
X_df.head()

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,...,14h,15h,16h,17h,18h,19h,20h,21h,22h,23h
0,5000,30,2,17,20,8,10,175,5000,110,...,0,0,0,0,0,0,1,0,0,0
1,10000,30,8,21,14,10,5,125,5000,31,...,1,0,0,0,0,0,0,0,0,0
2,18000,27,5,14,8,19,5,500,10000,4,...,0,0,0,0,0,0,0,0,0,0
3,7000,56,5,17,17,8,10,87,1000,56,...,0,0,0,1,0,0,0,0,0,0
4,8150,30,8,16,11,11,4,42,815,41,...,0,0,0,0,0,0,0,0,0,0


In [6]:
outcomes_df = pd.read_pickle('outcomes_df.save')

In [12]:
all_predictors = np.array(X_df.columns)
print all_predictors

['0h' '10d' '10h' '11d' '11h' '12d' '12h' '13d' '13h' '14d' '14h' '15d'
 '15h' '16d' '16h' '17d' '17h' '18d' '18h' '19d' '19h' '1d' '1h' '20d'
 '20h' '21d' '21h' '22d' '22h' '23d' '23h' '24d' '25d' '26d' '27d' '28d'
 '29d' '2d' '2h' '30d' '31d' '3d' '3h' '4d' '4h' '5d' '5h' '6d' '6h' '7d'
 '7h' '8d' '8h' '9d' '9h' 'Apr' 'Aug' 'Blurb_n_words' 'Body_n_words'
 'Campaign_days' 'Dec' 'Feb' 'Fri' 'Goal' 'Hour' 'Jan' 'Jul' 'Jun' 'Mar'
 'Max_reward' 'May' 'Median_reward' 'Min_reward' 'Mon' 'N_creator_backed'
 'N_figs' 'N_links' 'N_rewards' 'Nov' 'Oct' 'Reward_ship_days' 'Sat' 'Sep'
 'Ships_intn' 'Sun' 'Thu' 'Title_n_words' 'Tue' 'Wed' 'art' 'comics'
 'crafts' 'dance' 'design' 'fashion' 'film' 'food' 'games' 'journalism'
 'music' 'photography' 'publishing' 'technology' 'theater']


In [37]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = \
cross_validation.train_test_split(X_df, outcomes_df['Outcome'].values, test_size=0.25, random_state=0)

In [38]:
np.shape(X_train)

(99803, 104)

In [39]:
np.shape(y_train)

(99803,)

In [28]:
X_train.head(2)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,...,14h,15h,16h,17h,18h,19h,20h,21h,22h,23h
16096,4625,31,9,19,22,9,5,50,1000,132,...,0,0,0,0,0,0,0,0,1,0
76234,75000,43,9,21,20,14,10,137,5000,110,...,0,0,0,0,0,0,1,0,0,0


In [41]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=9)
clf.fit(X_train,y_train)
y_1 = clf.predict(X_test)

In [43]:
print 'accuracy = ', round(clf.score(X_test,y_test),4)

 accuracy =  0.7401


In [46]:
from sklearn import metrics
metrics.precision_recall_fscore_support(y_test,y_1)

(array([ 0.7471756 ,  0.72718769]),
 array([ 0.83360133,  0.61126599]),
 array([ 0.78802589,  0.66420693]),
 array([19279, 13989]))

In [49]:
feature_importances = clf.feature_importances_

In [50]:
feature_importance_is = np.argsort(feature_importances)

In [51]:
feature_importance_is_decending = feature_importance_is[-1::-1]

In [53]:
X_df.columns[feature_importance_is_decending][0:15]

Index([u'N_creator_backed', u'Goal', u'N_rewards', u'Body_n_words', u'N_links',
       u'music', u'Campaign_days', u'Max_reward', u'Median_reward',
       u'Reward_ship_days', u'N_figs', u'Min_reward', u'theater',
       u'publishing', u'dance'],
      dtype='object')

In [54]:
# write a script for showing gini importances

In [68]:
n_features = 104
n_top = 30

In [69]:
plt.rcParams.update({'font.size': 10})

fig = plt.figure()
plt.title("Feature importances")
plt.bar(range(n_top), feature_importances[feature_importance_is_decending[0:n_top]],
       color="b", align="center")
plt.xticks(range(n_top), X_df.columns[feature_importance_is_decending[0:n_top]],rotation='vertical')
plt.xlim([-1, n_top])
plt.xlabel('Feature')
plt.ylabel('Feature importance (Gini importance)')
plt.show()

In [70]:
# # run this later
# # try a cross validated random tree
# from sklearn import grid_search

# rf = RandomForestClassifier()
# parameters = {'n_estimators':[200, 300, 400],
#               'max_depth':[5, 7, 9]}

# # Warning: be sure your data is shuffled before using GridSearch!
# clf_grid = grid_search.GridSearchCV(rf, parameters)
# clf_grid.fit(X_train,y_train)

In [71]:
# also try min_samples_split
# mi_samples_leaf

In [72]:
# try excluding categories? 

In [73]:
# nicer plots -- update presentation
# add plots to site
# compare models -- logistic regression, svm